# Setup and Context

### Introduction

On November 27, 1895, Alfred Nobel signed his last will in Paris. When it was opened after his death, the will caused a lot of controversy, as Nobel had left much of his wealth for the establishment of a prize.

Alfred Nobel dictates that his entire remaining estate should be used to endow “prizes to those who, during the preceding year, have conferred the greatest benefit to humankind”.

Every year the Nobel Prize is given to scientists and scholars in the categories chemistry, literature, physics, physiology or medicine, economics, and peace. 

<img src=https://i.imgur.com/36pCx5Q.jpg>

Let's see what patterns we can find in the data of the past Nobel laureates. What can we learn about the Nobel prize and our world more generally?

### Upgrade plotly (only Google Colab Notebook)

Google Colab may not be running the latest version of plotly. If you're working in Google Colab, uncomment the line below, run the cell, and restart your notebook server. 

In [77]:
# %pip install --upgrade plotly

### Import Statements

In [78]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

### Notebook Presentation

In [79]:
pd.options.display.float_format = '{:,.2f}'.format

### Read the Data

In [80]:
df_data = pd.read_csv('nobel_prize_data.csv')

Caveats: The exact birth dates for Michael Houghton, Venkatraman Ramakrishnan, and Nadia Murad are unknown. I've substituted them with mid-year estimate of July 2nd. 


# Data Exploration & Cleaning

**Challenge**: Preliminary data exploration. 
* What is the shape of `df_data`? How many rows and columns?
* What are the column names?
* In which year was the Nobel prize first awarded?
* Which year is the latest year included in the dataset?

In [81]:
print(df_data.shape)
# shape is 962 rows, 16 columns

print(df_data.columns)
print(df_data.dtypes)
# col names:  'year', 'category', 'prize', 'motivation', 'prize_share',
    #    'laureate_type', 'full_name', 'birth_date', 'birth_city',
    #    'birth_country', 'birth_country_current', 'sex', 'organization_name',
    #    'organization_city', 'organization_country', 'ISO'
# objects except year

print(df_data.year.min())
# 1901 was first Nobel prize

print(df_data.year.max())
# 2020 is latest year reported



(962, 16)
Index(['year', 'category', 'prize', 'motivation', 'prize_share',
       'laureate_type', 'full_name', 'birth_date', 'birth_city',
       'birth_country', 'birth_country_current', 'sex', 'organization_name',
       'organization_city', 'organization_country', 'ISO'],
      dtype='object')
year                      int64
category                 object
prize                    object
motivation               object
prize_share              object
laureate_type            object
full_name                object
birth_date               object
birth_city               object
birth_country            object
birth_country_current    object
sex                      object
organization_name        object
organization_city        object
organization_country     object
ISO                      object
dtype: object
1901
2020


**Challange**: 
* Are there any duplicate values in the dataset?
* Are there NaN values in the dataset?
* Which columns tend to have NaN values?
* How many NaN values are there per column? 
* Why do these columns have NaN values?  

### Check for Duplicates

In [82]:
print(df_data[df_data.duplicated()])
# empty - no duplicate data


Empty DataFrame
Columns: [year, category, prize, motivation, prize_share, laureate_type, full_name, birth_date, birth_city, birth_country, birth_country_current, sex, organization_name, organization_city, organization_country, ISO]
Index: []


### Check for NaN Values

In [83]:
print(df_data[df_data.isna()])
# nan columns: year, category, prize, motivation, prize_share, laureate_type, full_name,
# birth_date, birth_city, birth_country, birth_country_current, sex

     year category prize motivation prize_share laureate_type full_name  \
0     NaN      NaN   NaN        NaN         NaN           NaN       NaN   
1     NaN      NaN   NaN        NaN         NaN           NaN       NaN   
2     NaN      NaN   NaN        NaN         NaN           NaN       NaN   
3     NaN      NaN   NaN        NaN         NaN           NaN       NaN   
4     NaN      NaN   NaN        NaN         NaN           NaN       NaN   
..    ...      ...   ...        ...         ...           ...       ...   
957   NaN      NaN   NaN        NaN         NaN           NaN       NaN   
958   NaN      NaN   NaN        NaN         NaN           NaN       NaN   
959   NaN      NaN   NaN        NaN         NaN           NaN       NaN   
960   NaN      NaN   NaN        NaN         NaN           NaN       NaN   
961   NaN      NaN   NaN        NaN         NaN           NaN       NaN   

    birth_date birth_city birth_country birth_country_current  sex  \
0          NaN        NaN    

In [84]:
print(df_data.isna().sum())
# 962 nan values per column -- WRONG, .count() will count both True and False values of .isna() !
# CORRECT: NaN values for motivation, several birth data columns, sex, organization info

year                       0
category                   0
prize                      0
motivation                88
prize_share                0
laureate_type              0
full_name                  0
birth_date                28
birth_city                31
birth_country             28
birth_country_current     28
sex                       28
organization_name        255
organization_city        255
organization_country     254
ISO                       28
dtype: int64


In [85]:
column_selection = ['full_name', 'motivation', 'birth_date', 'birth_city', 'birth_country', 'birth_country_current',
               'sex', 'organization_name', 'organization_city', 'organization_country']

df_data.loc[df_data.organization_name.isna()][column_selection]
# some have Nan values due to:
# birth, sex info as they are organizations
# organiation info if they are an individual 


full_name  \
1                       Sully Prudhomme   
3                        Frédéric Passy   
4                     Jean Henry Dunant   
7    Christian Matthias Theodor Mommsen   
9                  Charles Albert Gobat   
..                                  ...   
932                         Nadia Murad   
942                        Peter Handke   
946                      Abiy Ahmed Ali   
954                        Louise Glück   
958          World Food Programme (WFP)   

                                            motivation  birth_date  \
1    "in special recognition of his poetic composit...  1839-03-16   
3                                                  NaN  1822-05-20   
4                                                  NaN  1828-05-08   
7    "the greatest living master of the art of hist...  1817-11-30   
9                                                  NaN  1843-05-21   
..                                                 ...         ...   
932  “for their efforts to end the use of sexual vi...  1993-07-02   
942  “for an influential work that with linguistic ...  1942-12-06   
946  “for his efforts to achieve peace and internat...  1976-08-15   
954  “for her unmistakable poetic voice that with a...  1943-04-22   
958  “for its efforts to combat hunger, for its con...         NaN   

       birth_city             birth_country     birth_country_current     sex  \
1           Paris                    France                    France    Male   
3           Paris                    France                    France    Male   
4          Geneva               Switzerland               Switzerland    Male   
7         Garding       Schleswig (Germany)                   Germany    Male   
9        Tramelan               Switzerland               Switzerland    Male   
..            ...                       ...                       ...     ...   
932          Kojo                      Iraq                      Iraq  Female   
942       Griffen                   Austria                   Austria    Male   
946      Beshasha                  Ethiopia                  Ethiopia    Male   
954  New York, NY  United States of America  United States of America  Female   
958           NaN                       NaN                       NaN     NaN   

    organization_name organization_city organization_country  
1                 NaN               NaN                  NaN  
3                 NaN               NaN                  NaN  
4                 NaN               NaN                  NaN  
7                 NaN               NaN                  NaN  
9                 NaN               NaN                  NaN  
..                ...               ...                  ...  
932               NaN               NaN                  NaN  
942               NaN               NaN                  NaN  
946               NaN               NaN                  NaN  
954               NaN               NaN                  NaN  
958               NaN               NaN                  NaN  

[255 rows x 10 columns]

### Type Conversions

**Challenge**: 
* Convert the `birth_date` column to Pandas `Datetime` objects
* Add a Column called `share_pct` which has the laureates' share as a percentage in the form of a floating-point number.

#### Convert Year and Birth Date to Datetime

In [86]:
# df_data.year = pd.to_datetime(df_data.year)
# print(df_data.year.dtype)

df_data.birth_date = pd.to_datetime(df_data.birth_date)
print(df_data.birth_date.dtype)

datetime64[ns]


#### Add a Column with the Prize Share as a Percentage

In [87]:
# df_data.columns
df_data.head()

separated_values = df_data.prize_share.str.split('/', expand=True)
numerator, denominator = pd.to_numeric(separated_values[0]), pd.to_numeric(separated_values[1])
df_data['share_pct'] = numerator/denominator

df_data.sample(5)

year    category                                           prize  \
23   1904    Medicine  The Nobel Prize in Physiology or Medicine 1904   
573  1987   Chemistry               The Nobel Prize in Chemistry 1987   
617  1991    Medicine  The Nobel Prize in Physiology or Medicine 1991   
11   1902     Physics                 The Nobel Prize in Physics 1902   
880  2014  Literature              The Nobel Prize in Literature 2014   

                                            motivation prize_share  \
23   "in recognition of his work on the physiology ...         1/1   
573  "for their development and use of molecules wi...         1/3   
617  "for their discoveries concerning the function...         1/2   
11   "in recognition of the extraordinary service t...         1/2   
880  "for the art of memory with which he has evoke...         1/1   

    laureate_type               full_name birth_date birth_city birth_country  \
23     Individual   Ivan Petrovich Pavlov 1849-09-14     Ryazan        Russia   
573    Individual         Jean-Marie Lehn 1939-09-30    Rosheim        France   
617    Individual             Erwin Neher 1944-03-20  Landsberg       Germany   
11     Individual  Hendrik Antoon Lorentz 1853-07-18     Arnhem   Netherlands   
880    Individual         Patrick Modiano 1945-07-30      Paris        France   

    birth_country_current   sex         organization_name organization_city  \
23                 Russia  Male  Military Medical Academy    St. Petersburg   
573                France  Male  Université Louis Pasteur        Strasbourg   
617               Germany  Male       Max-Planck-Institut         Göttingen   
11            Netherlands  Male         Leiden University            Leiden   
880                France  Male                       NaN               NaN   

    organization_country  ISO  share_pct  
23                Russia  RUS       1.00  
573               France  FRA       0.33  
617              Germany  DEU       0.50  
11           Netherlands  NLD       0.50  
880                  NaN  FRA       1.00

# Plotly Donut Chart: Percentage of Male vs. Female Laureates

**Challenge**: Create a [donut chart using plotly](https://plotly.com/python/pie-charts/) which shows how many prizes went to men compared to how many prizes went to women. What percentage of all the prizes went to women?

In [88]:
import plotly.graph_objects as go

In [89]:
# labels = df_data.sex.unique()
# print(labels)

biology = df_data.sex.value_counts()
print(biology)

fig = px.pie(labels=biology.index, values=biology.values, title='% of Male vs. Female Nobel Prize Winners', hole=.4) 
fig.update_traces(textposition='inside', textfont_size=15, textinfo='percent')

fig.show()


sex
Male      876
Female     58
Name: count, dtype: int64


# Who were the first 3 Women to Win the Nobel Prize?

**Challenge**: 
* What are the names of the first 3 female Nobel laureates? 
* What did the win the prize for? 
* What do you see in their `birth_country`? Were they part of an organisation?

In [94]:
first_women = df_data[df_data.sex == 'Female'].sort_values('year', ascending=True)
first_women[:3]

year    category                               prize  \
18  1903     Physics     The Nobel Prize in Physics 1903   
29  1905       Peace          The Nobel Peace Prize 1905   
51  1909  Literature  The Nobel Prize in Literature 1909   

                                           motivation prize_share  \
18  "in recognition of the extraordinary services ...         1/4   
29                                                NaN         1/1   
51  "in appreciation of the lofty idealism, vivid ...         1/1   

   laureate_type                                          full_name  \
18    Individual                        Marie Curie, née Sklodowska   
29    Individual  Baroness Bertha Sophie Felicita von Suttner, n...   
51    Individual                      Selma Ottilia Lovisa Lagerlöf   

   birth_date birth_city                     birth_country  \
18 1867-11-07     Warsaw           Russian Empire (Poland)   
29 1843-06-09     Prague  Austrian Empire (Czech Republic)   
51 1858-11-20   Mårbacka                            Sweden   

   birth_country_current     sex organization_name organization_city  \
18                Poland  Female               NaN               NaN   
29        Czech Republic  Female               NaN               NaN   
51                Sweden  Female               NaN               NaN   

   organization_country  ISO  share_pct  
18                  NaN  POL       0.25  
29                  NaN  CZE       1.00  
51                  NaN  SWE       1.00

In [ ]:
# first 3 women were Marie Curie, Baroness Bertha Sophie Felicitya von..., Selma Ottilia Lovisa
# won prizes for physics, peace, and litearture
# birth countries: poland, czech republic, sweden

# Find the Repeat Winners

**Challenge**: Did some people get a Nobel Prize more than once? If so, who were they? 

In [110]:
is_winner = df_data.duplicated(subset=['full_name'], keep=False) # returns True value for duplicate winners
multiple_winners = df_data[is_winner].sort_values('full_name')
print(multiple_winners.full_name.value_counts())

multiple_winners

full_name
Comité international de la Croix Rouge (International Committee of the Red Cross)    3
Frederick Sanger                                                                     2
John Bardeen                                                                         2
Linus Carl Pauling                                                                   2
Marie Curie, née Sklodowska                                                          2
Office of the United Nations High Commissioner for Refugees (UNHCR)                  2
Name: count, dtype: int64


year   category                              prize  \
89   1917      Peace         The Nobel Peace Prize 1917   
215  1944      Peace         The Nobel Peace Prize 1944   
348  1963      Peace         The Nobel Peace Prize 1963   
306  1958  Chemistry  The Nobel Prize in Chemistry 1958   
505  1980  Chemistry  The Nobel Prize in Chemistry 1980   
297  1956    Physics    The Nobel Prize in Physics 1956   
424  1972    Physics    The Nobel Prize in Physics 1972   
278  1954  Chemistry  The Nobel Prize in Chemistry 1954   
340  1962      Peace         The Nobel Peace Prize 1962   
18   1903    Physics    The Nobel Prize in Physics 1903   
62   1911  Chemistry  The Nobel Prize in Chemistry 1911   
283  1954      Peace         The Nobel Peace Prize 1954   
523  1981      Peace         The Nobel Peace Prize 1981   

                                            motivation prize_share  \
89                                                 NaN         1/1   
215                                                NaN         1/1   
348                                                NaN         1/2   
306  "for his work on the structure of proteins, es...         1/1   
505  "for their contributions concerning the determ...         1/4   
297  "for their researches on semiconductors and th...         1/3   
424  "for their jointly developed theory of superco...         1/3   
278  "for his research into the nature of the chemi...         1/1   
340                                                NaN         1/1   
18   "in recognition of the extraordinary services ...         1/4   
62   "in recognition of her services to the advance...         1/1   
283                                                NaN         1/1   
523                                                NaN         1/1   

    laureate_type                                          full_name  \
89   Organization  Comité international de la Croix Rouge (Intern...   
215  Organization  Comité international de la Croix Rouge (Intern...   
348  Organization  Comité international de la Croix Rouge (Intern...   
306    Individual                                   Frederick Sanger   
505    Individual                                   Frederick Sanger   
297    Individual                                       John Bardeen   
424    Individual                                       John Bardeen   
278    Individual                                 Linus Carl Pauling   
340    Individual                                 Linus Carl Pauling   
18     Individual                        Marie Curie, née Sklodowska   
62     Individual                        Marie Curie, née Sklodowska   
283  Organization  Office of the United Nations High Commissioner...   
523  Organization  Office of the United Nations High Commissioner...   

    birth_date    birth_city             birth_country  \
89         NaT           NaN                       NaN   
215        NaT           NaN                       NaN   
348        NaT           NaN                       NaN   
306 1918-08-13     Rendcombe            United Kingdom   
505 1918-08-13     Rendcombe            United Kingdom   
297 1908-05-23   Madison, WI  United States of America   
424 1908-05-23   Madison, WI  United States of America   
278 1901-02-28  Portland, OR  United States of America   
340 1901-02-28  Portland, OR  United States of America   
18  1867-11-07        Warsaw   Russian Empire (Poland)   
62  1867-11-07        Warsaw   Russian Empire (Poland)   
283        NaT           NaN                       NaN   
523        NaT           NaN                       NaN   

        birth_country_current     sex  \
89                        NaN     NaN   
215                       NaN     NaN   
348                       NaN     NaN   
306            United Kingdom    Male   
505            United Kingdom    Male   
297  United States of America    Male   
424  United States of America    Male   
278  United States of America    Male   
340  U

# Number of Prizes per Category

**Challenge**: 
* In how many categories are prizes awarded? 
* Create a plotly bar chart with the number of prizes awarded by category. 
* Use the color scale called `Aggrnyl` to colour the chart, but don't show a color axis.
* Which category has the most number of prizes awarded? 
* Which category has the fewest number of prizes awarded? 

In [123]:
df_data.category.nunique()
# 6 categories

6

In [130]:
from matplotlib.pyplot import xlabel, ylabel


prizes_by_category = df_data.category.value_counts()

fig = px.bar(prizes_by_category, prizes_by_category.index, prizes_by_category.values,
             title='Nobel Peace Prizes By Category',
             color=prizes_by_category.values)
fig.update_layout(xaxis_title='Category', yaxis_title='Number of Prizes',
                  coloraxis_showscale=False)
fig.show()

**Challenge**: 
* When was the first prize in the field of Economics awarded?
* Who did the prize go to?

In [133]:
economics = df_data[df_data.category == 'Economics'].sort_values('year', ascending=True)
economics.head()
# first economics prize was 1969, awarded to Jan Tinbergen

year   category                                              prize  \
393  1969  Economics  The Sveriges Riksbank Prize in Economic Scienc...   
394  1969  Economics  The Sveriges Riksbank Prize in Economic Scienc...   
402  1970  Economics  The Sveriges Riksbank Prize in Economic Scienc...   
411  1971  Economics  The Sveriges Riksbank Prize in Economic Scienc...   
419  1972  Economics  The Sveriges Riksbank Prize in Economic Scienc...   

                                            motivation prize_share  \
393  "for having developed and applied dynamic mode...         1/2   
394  "for having developed and applied dynamic mode...         1/2   
402  "for the scientific work through which he has ...         1/1   
411  "for his empirically founded interpretation of...         1/1   
419  "for their pioneering contributions to general...         1/2   

    laureate_type          full_name birth_date birth_city  \
393    Individual      Jan Tinbergen 1903-04-12  the Hague   
394    Individual      Ragnar Frisch 1895-03-03       Oslo   
402    Individual  Paul A. Samuelson 1915-05-15   Gary, IN   
411    Individual      Simon Kuznets 1901-04-30      Pinsk   
419    Individual      John R. Hicks 1904-04-08    Warwick   

                birth_country     birth_country_current   sex  \
393               Netherlands               Netherlands  Male   
394                    Norway                    Norway  Male   
402  United States of America  United States of America  Male   
411  Russian Empire (Belarus)                   Belarus  Male   
419            United Kingdom            United Kingdom  Male   

                               organization_name organization_city  \
393          The Netherlands School of Economics         Rotterdam   
394                           University of Oslo              Oslo   
402  Massachusetts Institute of Technology (MIT)     Cambridge, MA   
411                           Harvard University     Cambridge, MA   
419                         University of Oxford            Oxford   

         organization_country  ISO  share_pct  
393               Netherlands  NLD       0.50  
394                    Norway  NOR       0.50  
402  United States of America  USA       1.00  
411  United States of America  BLR       1.00  
419            United Kingdom  GBR       0.50

# Male and Female Winners by Category

**Challenge**: Create a [plotly bar chart](https://plotly.com/python/bar-charts/) that shows the split between men and women by category. 
* Hover over the bar chart. How many prizes went to women in Literature compared to Physics?

<img src=https://i.imgur.com/od8TfOp.png width=650>

In [136]:
sex_counts_by_category = df_data.groupby(['category', 'sex'],
                                         as_index=False).agg(
                                             {'prize': pd.Series.count}
                                         )

sex_counts_by_category.sort_values('prize', ascending=False, inplace=True)

fig = px.bar(x = sex_counts_by_category.category,
             y = sex_counts_by_category.prize,
             color = sex_counts_by_category.sex,
             title = 'Male and Female Nobel Prize Receipients by Category')

fig.update_layout(xaxis_title = 'Category',
                  yaxis_title = 'Number of Prizes')

fig.show()


# Number of Prizes Awarded Over Time

**Challenge**: Are more prizes awarded recently than when the prize was first created? Show the trend in awards visually. 
* Count the number of prizes awarded every year. 
* Create a 5 year rolling average of the number of prizes (Hint: see previous lessons analysing Google Trends).
* Using Matplotlib superimpose the rolling average on a scatter plot.
* Show a tick mark on the x-axis for every 5 years from 1900 to 2020. (Hint: you'll need to use NumPy). 

<img src=https://i.imgur.com/4jqYuWC.png width=650>

* Use the [named colours](https://matplotlib.org/3.1.0/gallery/color/named_colors.html) to draw the data points in `dogerblue` while the rolling average is coloured in `crimson`. 

<img src=https://i.imgur.com/u3RlcJn.png width=350>

* Looking at the chart, did the first and second world wars have an impact on the number of prizes being given out? 
* What could be the reason for the trend in the chart?


# Are More Prizes Shared Than Before?

**Challenge**: Investigate if more prizes are shared than before. 

* Calculate the average prize share of the winners on a year by year basis.
* Calculate the 5 year rolling average of the percentage share.
* Copy-paste the cell from the chart you created above.
* Modify the code to add a secondary axis to your Matplotlib chart.
* Plot the rolling average of the prize share on this chart. 
* See if you can invert the secondary y-axis to make the relationship even more clear. 

# The Countries with the Most Nobel Prizes

**Challenge**: 
* Create a Pandas DataFrame called `top20_countries` that has the two columns. The `prize` column should contain the total number of prizes won. 

<img src=https://i.imgur.com/6HM8rfB.png width=350>

* Is it best to use `birth_country`, `birth_country_current` or `organization_country`? 
* What are some potential problems when using `birth_country` or any of the others? Which column is the least problematic? 
* Then use plotly to create a horizontal bar chart showing the number of prizes won by each country. Here's what you're after:

<img src=https://i.imgur.com/agcJdRS.png width=750>

* What is the ranking for the top 20 countries in terms of the number of prizes?

# Use a Choropleth Map to Show the Number of Prizes Won by Country

* Create this choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/):

<img src=https://i.imgur.com/s4lqYZH.png>

* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 

Hint: You'll need to use a 3 letter country code for each country. 


# In Which Categories are the Different Countries Winning Prizes? 

**Challenge**: See if you can divide up the plotly bar chart you created above to show the which categories made up the total number of prizes. Here's what you're aiming for:

<img src=https://i.imgur.com/iGaIKCL.png>

* In which category are Germany and Japan the weakest compared to the United States?
* In which category does Germany have more prizes than the UK?
* In which categories does France have more prizes than Germany?
* Which category makes up most of Australia's nobel prizes?
* Which category makes up half of the prizes in the Netherlands?
* Does the United States have more prizes in Economics than all of France? What about in Physics or Medicine?


The hard part is preparing the data for this chart! 


*Hint*: Take a two-step approach. The first step is grouping the data by country and category. Then you can create a DataFrame that looks something like this:

<img src=https://i.imgur.com/VKjzKa1.png width=450>


### Number of Prizes Won by Each Country Over Time

* When did the United States eclipse every other country in terms of the number of prizes won? 
* Which country or countries were leading previously?
* Calculate the cumulative number of prizes won by each country in every year. Again, use the `birth_country_current` of the winner to calculate this. 
* Create a [plotly line chart](https://plotly.com/python/line-charts/) where each country is a coloured line. 

# What are the Top Research Organisations?

**Challenge**: Create a bar chart showing the organisations affiliated with the Nobel laureates. It should looks something like this:

<img src=https://i.imgur.com/zZihj2p.png width=600>

* Which organisations make up the top 20?
* How many Nobel prize winners are affiliated with the University of Chicago and Harvard University?

# Which Cities Make the Most Discoveries? 

Where do major discoveries take place?  

**Challenge**: 
* Create another plotly bar chart graphing the top 20 organisation cities of the research institutions associated with a Nobel laureate. 
* Where is the number one hotspot for discoveries in the world?
* Which city in Europe has had the most discoveries?

# Where are Nobel Laureates Born? Chart the Laureate Birth Cities 

**Challenge**: 
* Create a plotly bar chart graphing the top 20 birth cities of Nobel laureates. 
* Use a named colour scale called `Plasma` for the chart.
* What percentage of the United States prizes came from Nobel laureates born in New York? 
* How many Nobel laureates were born in London, Paris and Vienna? 
* Out of the top 5 cities, how many are in the United States?


# Plotly Sunburst Chart: Combine Country, City, and Organisation

**Challenge**: 

* Create a DataFrame that groups the number of prizes by organisation. 
* Then use the [plotly documentation to create a sunburst chart](https://plotly.com/python/sunburst-charts/)
* Click around in your chart, what do you notice about Germany and France? 


Here's what you're aiming for:

<img src=https://i.imgur.com/cemX4m5.png width=300>



# Patterns in the Laureate Age at the Time of the Award

How Old Are the Laureates When the Win the Prize?

**Challenge**: Calculate the age of the laureate in the year of the ceremony and add this as a column called `winning_age` to the `df_data` DataFrame. Hint: you can use [this](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to help you. 



### Who were the oldest and youngest winners?

**Challenge**: 
* What are the names of the youngest and oldest Nobel laureate? 
* What did they win the prize for?
* What is the average age of a winner?
* 75% of laureates are younger than what age when they receive the prize?
* Use Seaborn to [create histogram](https://seaborn.pydata.org/generated/seaborn.histplot.html) to visualise the distribution of laureate age at the time of winning. Experiment with the number of `bins` to see how the visualisation changes.

### Descriptive Statistics for the Laureate Age at Time of Award

* Calculate the descriptive statistics for the age at the time of the award. 
* Then visualise the distribution in the form of a histogram using [Seaborn's .histplot() function](https://seaborn.pydata.org/generated/seaborn.histplot.html).
* Experiment with the `bin` size. Try 10, 20, 30, and 50.  

### Age at Time of Award throughout History

Are Nobel laureates being nominated later in life than before? Have the ages of laureates at the time of the award increased or decreased over time?

**Challenge**

* Use Seaborn to [create a .regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html?highlight=regplot#seaborn.regplot) with a trendline.
* Set the `lowess` parameter to `True` to show a moving average of the linear fit.
* According to the best fit line, how old were Nobel laureates in the years 1900-1940 when they were awarded the prize?
* According to the best fit line, what age would it predict for a Nobel laureate in 2020?


### Winning Age Across the Nobel Prize Categories

How does the age of laureates vary by category? 

* Use Seaborn's [`.boxplot()`](https://seaborn.pydata.org/generated/seaborn.boxplot.html?highlight=boxplot#seaborn.boxplot) to show how the mean, quartiles, max, and minimum values vary across categories. Which category has the longest "whiskers"? 
* In which prize category are the average winners the oldest?
* In which prize category are the average winners the youngest?

**Challenge**
* Now use Seaborn's [`.lmplot()`](https://seaborn.pydata.org/generated/seaborn.lmplot.html?highlight=lmplot#seaborn.lmplot) and the `row` parameter to create 6 separate charts for each prize category. Again set `lowess` to `True`.
* What are the winning age trends in each category? 
* Which category has the age trending up and which category has the age trending down? 
* Is this `.lmplot()` telling a different story from the `.boxplot()`?
* Create another chart with Seaborn. This time use `.lmplot()` to put all 6 categories on the same chart using the `hue` parameter. 
